# [Tensorflow - Help Protect the Great Barrier Reef](https://www.kaggle.com/c/tensorflow-great-barrier-reef)
> Detect crown-of-thorns starfish in underwater image data

<img src="https://storage.googleapis.com/kaggle-competitions/kaggle/31703/logos/header.png?t=2021-10-29-00-30-04">

## 当たり前にやられていること
- [ ] Dataの修正・追加
    - [ ] サイズが大きすぎる・小さすぎるbboxを無視（実装）
    - [ ] 手作業でbboxを追加（解説）
- [ ] Data Augmentation
    - [ ] Albumentationsライブラリを使ったaugmentation（RandomSizedCrop, HueSaturationValue, RandomBrightnessContrast, ToGray, HorizontalFlip, VerticalFlip, Cutout, etc）
    - [ ] mixup（画像の合成。実装）
    - [ ] cutmix（cutout+mixup: cutoutした部分に別画像を合成。実装 ）
    - [ ] ジグソーパズルによる画像生成（既述）
- [ ] アーキテクチャの選択
    - [ ] YOLO（コンペ参加直後に触っていた。v5はライセンスの問題で使用禁止に。単独ではおそらく最高精度が出せるモデルだった）
    - [ ] EfficientDet（YOLOv5が禁止になってからはひたすらD5を中心にEfficientDetで実験していた。EfficientNetの考え方を取り入れた物体検出モデル。実装）
    - [ ] 他は試してないが、DetectorRSやUniverseNetが良いなどの報告あり
- [ ] 高解像度で学習
    - [ ] リサイズを行わず1024 x 1024の画像で学習（Colab Proではbatch size 1でギリギリCUDA out of memoryを回避できる）
- [x] TTA（テストデータもaugmentation。実装）
- [ ] Pseudo Labeling (テーブルデータでもお馴染み、テストデータを予測し確信度の高いラベルのみ訓練データに取り入れて再予測。実装)
- [ ] Ensemble (精度を求めるKaggleではWBFが強い場合が多そう。実装, 解説)
    - [ ] NMS（IoUがある閾値を超えて重なっているbboxの集合から、スコアが最大のbboxを残して、それ以外を除去）
    - [ ] SoftNMS（IoU閾値を超えたbboxを残しつつ、スコアが最大のbbox以外も除去せず、スコアを割り引いて残す）
    - [ ] NMW (重なりあったbboxをスコアとIoUで重み付けして足し合わせることで、1つの新たなbboxを作り出す)
    - [ ] WBF（検出されたモデルの数が少ないbboxほどスコアを下げることで、少数のモデルだけで検出されたbboxをスコアで足切りする）

アイデア

- ヒトデが映ってない画像も学習データにしたらダメか？
    - 全部使うとほとんどヒトデが映ってない、と判定しそう
        - 同じ枚数(5k)だったらいいかも？
- モデルを変える
    - YoloX
    - EfficientDet
    - FasterRCNN
    - DETR
- nofair tracking
- コントラスト均等化する
- Augmentationをしない
    - メモリ節約のため
- Adamを使う

わかったこと
- yolov5について
    - 学習にはtrain.pyという元々あるコードを使っている
    - それに引数として学習データのパスやハイパーパラメータの情報が記述された.yamlファイルを与えている

やる事
- yolov5とは何か調査
    - 何を学習するのか？
    - 精度とかはどうやって出すのか？
    - 見つけたいものが映ってないデータは学習に含めてよいのか？
- いらない部分を削ってシンプルにする
- W&Bにログインして学習経過のグラフなどを見る
    - ~というか、細かい処理とかwandbに登録してこっちでやってそう~
- クロスバリデーションするモデル5個作ってアンサンブル

## ファイル構造

working/labels/以下にすべてのファイルのlabel.txtが入っている

imagesを同じ

# 🛠 Install Libraries

In [ ]:
!pip install -qU wandb
!pip install -qU bbox-utility # check https://github.com/awsaf49/bbox for source code

# 📚 Import Libraries

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import glob

import shutil
import sys
sys.path.append('../input/tensorflow-great-barrier-reef')

from joblib import Parallel, delayed

from IPython.display import display, HTML

from matplotlib import animation, rc
rc('animation', html='jshtml')

# for DA
from torch.utils.data import DataLoader, Dataset
import torch.utils.data as Data
import ast #?
from fastprogress.fastprogress import master_bar, progress_bar #?

# 📌 Key-Points
* 提供されているpython時系列APIを使用して予測を送信する必要があります。これにより、このコンテストは以前のオブジェクト検出コンテストとは異なります。
* 各予測行には、画像のすべての境界ボックスを含める必要があります。提出はフォーマットもCOCOのようです。これは`[x_min、y_min、幅、高さ]`を意味します
* CopmetitionメトリックF2は、ヒトデを見逃すことがほとんどないことを保証するために、いくつかの誤検知（FP）を許容します。つまり、誤検知（FN）は、誤検知（FP）よりも重要です。
$$F2 = 5 \cdot \frac{precision \cdot recall}{4\cdot precision + recall}$$

# ⭐ WandB

In [ ]:
import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("wandb_team_iforine")
    wandb.login(key=api_key)
    anonymous = None
except:
    wandb.login(anonymous='must')
    print('To use your W&B account,\nGo to Add-ons -> Secrets and provide your W&B access token. Use the Label name as WANDB. \nGet your W&B access token from here: https://wandb.ai/authorize')

# 📖 Meta Data
* `train_images/` - Folder containing training set photos of the form `video_{video_id}/{video_frame}.jpg`.

* `[train/test].csv` - 画像のメタデータです。他のテストファイルと同様に、テストのメタデータデータのほとんどは、提出時にしかノートブックに表示されません。ダウンロードできるのは最初の数行だけです。

* `video_id` - 画像が含まれるビデオのID番号。ビデオIDは意味のある順序ではありません。
* `video_frame` - 映像内の画像のフレーム番号です。ダイバーが浮上したときからフレーム番号にずれが生じることがあります。
* `sequence` - ID of a gap-free subset of a given video. The sequence ids are not meaningfully ordered.指定されたビデオのギャップフリー部分集合のID。シーケンスIDは意味のある順序ではありません。
* `sequence_frame` - 指定されたシーケンス内のフレーム番号。
* `image_id` - ID code for the image, in the format `{video_id}-{video_frame}`
* `annotations` - The bounding boxes of any starfish detections in a string format that can be evaluated directly with Python. Does not use the same format as the predictions you will submit. Not available in test.csv. A bounding box is described by the pixel coordinate `(x_min, y_min)` of its lower left corner within the image together with its `width` and `height` in pixels --> (COCO format).Pythonで直接評価可能な文字列形式のヒトデ検出のバウンディングボックス。提出する予測値と同じフォーマットではありません。test.csvでは利用できません。バウンディングボックスは、画像内の左下隅のピクセル座標 `(x_min, y_min)` と、ピクセル単位の `width` と `height` で記述される --> (COCO 形式)。

In [ ]:
FOLD      = 4 # which fold to train
DIM       = 2016
MODEL     = 'yolov5s'
BATCH     = 4
EPOCHS    = 10
OPTIM     = 'Adam'
AUG       = 'HE'

PROJECT   = 'iforine/great-barrier-reef-public' # w&b in yolov5
NAME      = f'{MODEL}-dim{DIM}-fold{FOLD}-bat{BATCH}-opt{OPTIM}-aug{AUG}-epch{EPOCHS}-addNoCot' # w&b for yolov5

REMOVE_NOBBOX = False # remove images with no bbox
ADD_NOBBOX = True # bboxのある画像と同じ枚数分bboxの無い画像を学習データに加える
ROOT_DIR  = '/kaggle/input/tensorflow-great-barrier-reef/'
IMAGE_DIR = '/kaggle/working/images' # directory to save images
LABEL_DIR = '/kaggle/working/labels' # directory to save labels

WORKER = 4 # よくわかってない。スレッドの数とか？

np.random.seed(42)

## Create Directories

In [ ]:
!mkdir -p {IMAGE_DIR}
!mkdir -p {LABEL_DIR}

## Get Paths

In [ ]:
pd.set_option('display.max_columns', 10)

pandas.eval()

様々なバックエンドを使用して、Python式を文字列として評価します。

In [ ]:
# Train Data
df = pd.read_csv(f'{ROOT_DIR}/train.csv')
df['old_image_path'] = f'{ROOT_DIR}/train_images/video_'+df.video_id.astype(str)+'/'+df.video_frame.astype(str)+'.jpg'
df['image_path']  = f'{IMAGE_DIR}/'+df.image_id+'.jpg' # '/kaggle/working/images'
df['label_path']  = f'{LABEL_DIR}/'+df.image_id+'.txt' # '/kaggle/working/labels'
df['annotations'] = df['annotations'].progress_apply(eval) # apply(各要素に関数を適用する)の進捗を表示する。evalは何？
display(df.head(100))

## Number of BBoxes
> Nearly 80% images are without any bbox.

In [ ]:
df['num_bbox'] = df['annotations'].progress_apply(lambda x: len(x))
data = (df.num_bbox>0).value_counts(normalize=True)*100 # ユニークな要素の出現頻度を算出。normalize=Trueにすると合計が1になるように正規化される(割合になる)
print(f"No BBox: {data[0]:0.2f}% | With BBox: {data[1]:0.2f}%")

`Error displaying widget: model not found`

何らかの原因でprogressを出せないのかも。モデルがない、とはどういうこと？

実際にbboxの存在する画像を見てみる

# 🧹 Clean Data
* In this notebook, we use only **bboxed-images** (`~5k`). We can use all `~23K` images for train but most of them don't have any labels. So it would be easier to carry out experiments using only **bboxed images**.
* このノートブックでは、bboxed-images（〜5k）のみを使用します。trainには約23Kの画像をすべて使用できますが、ほとんどの画像にはラベルがありません。したがって、bboxed画像のみを使用して実験を実行する方が簡単です

In [ ]:
if REMOVE_NOBBOX:
    df = df.query("num_bbox>0") # df[df['num_bbox'] > 0]と同等。直観的で便利だ・・・

In [ ]:
# 背景画像を全体の10%含める
if ADD_NOBBOX:
    df = pd.concat([df.query("num_bbox>0"), df.query("num_bbox==0").sample(int(len(df.query("num_bbox>0")) * 0.1))])

# ✏️ Write Images
* We need to copy the Images to Current Directory(`/kaggle/working`) as `/kaggle/input` doesn't have **write access** which is needed for **YOLOv5**.
* We can make this process faster using **Joblib** which uses **Parallel** computing.

* / kaggle / inputにはYOLOv5に必要な書き込みアクセス権がないため、イメージを現在のディレクトリ（/ kaggle / working）にコピーする必要があります。
* この処理を高速化するには、**並列**計算を利用する**Joblib**を使用します。

shutil.copyfile(src, dst, *, follow_symlinks=True)

src という名前のファイルの内容 (メタデータを含まない) を dst という名前のファイルにコピーし、最も効率的な方法で dst を返します。 src と dst は path-like object または文字列でパス名を指定します。

In [ ]:
def make_copy(row):
    shutil.copyfile(row.old_image_path, row.image_path)
    return

並列処理

```
joblib.Parallel(<Parallelへの引数>)(
    joblib.delayed(<実行する関数>)(<関数への引数>) for 変数名 in イテラブル
)
```

iterrows()メソッドを使うと、1行ずつ、インデックス名（行名）とその行のデータ（pandas.Series型）のタプル(index, Series)を取得できる。

In [ ]:
image_paths = df.old_image_path.tolist()
_ = Parallel(n_jobs=-1, backend='threading')(delayed(make_copy)(row) for _, row in tqdm(df.iterrows(), total=len(df)))

# 🔨 Helper

In [ ]:
# check https://github.com/awsaf49/bbox for source code of following utility functions
# 作者が作ったヘルパー関数
from bbox.utils import coco2yolo, coco2voc, voc2yolo
from bbox.utils import draw_bboxes, load_image
from bbox.utils import clip_bbox, str2annot, annot2str

# bboxをリストにして返す
def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

# rowに幅と高さの列を追加する
def get_imgsize(row):
    row['width'], row['height'] = imagesize.get(row['image_path']) # このimagesizeって何？
    return row

np.random.seed(32)
colors = [(np.random.randint(255), np.random.randint(255), np.random.randint(255))\
          for idx in range(1)]

## Create BBox

In [ ]:
# annotionsからbboxes列を作成
df['bboxes'] = df.annotations.progress_apply(get_bbox)
df.head(2)

## Get Image-Size
> All Images have same dimension, [Width, Height] =  `[1280, 720]`

In [ ]:
df['width']  = 1280
df['height'] = 720
display(df.head(2))

# 🏷️ Create Labels
We need to export our labels to **YOLO** format, with one `*.txt` file per image (if no objects in image, no `*.txt` file is required). The *.txt file specifications are:

* One row per object
* Each row is class `[x_center, y_center, width, height]` format.
* Box coordinates must be in **normalized** `xywh` format (from `0 - 1`). If your boxes are in pixels, divide `x_center` and `width` by `image width`, and `y_center` and `height` by `image height`.
* Class numbers are **zero-indexed** (start from `0`).

> Competition bbox format is **COCO** hence `[x_min, y_min, width, height]`. So, we need to convert form **COCO** to **YOLO** format.

各画像に対して.txtファイルを作ってYOLOに対応する形式にする

* オブジェクト一つにつき1行
* 各行以下のフォーマット`[x_center, y_center, width, height]`
* box座標は**0-1で正規化された**xywhフォーマット。ピクセル単位の場合は、 `x_center` と `width` を `image width` で割って、 `y_center` と `height` を `image height` で割ってください。
* クラス番号は **0から始まるゼロ・インデックス** です。

> コンペのbbox形式はCOCOであるため、[x_min、y_min、width、height]です。したがって、フォームCOCOをYOLO形式に変換する必要があります。

In [ ]:
pd.set_option('display.max_columns', 100) # 列が多いと省略されるのを防ぐ

これがラベル？3のとこにはbboxの数が入る

coco => **[xmin, ymin, w, h]**

voc  => **[xmin, ymin, xmax, ymax]**

yolo => **[xmid, ymid, w, h]** (normalized)

```
def clip_bbox(bboxes_voc, height=720, width=1280):

    Clip bounding boxes to image boundaries.
    Args:
        bboxes_voc (np.ndarray): bboxes in [xmin, ymin, xmax, ymax] format.
        height (int, optional): height of bbox. Defaults to 720.
        width (int, optional): width of bbox. Defaults to 1280.
    Returns:
        np.ndarray : clipped bboxes in [xmin, ymin, xmax, ymax] format.
```

## label.txtを作成

In [ ]:
cnt = 0
all_bboxes = []
bboxes_info = []
for row_idx in tqdm(range(df.shape[0])):
    row = df.iloc[row_idx]
    image_height = row.height
    image_width  = row.width
    bboxes_coco  = np.array(row.bboxes).astype(np.float32).copy() #bboxesをnumpy形式に変換
    num_bbox     = len(bboxes_coco)
    names        = ['cots']*num_bbox
    labels       = np.array([0]*num_bbox)[..., None].astype(str) # 次元を++(リストからshape:(N, 1)の行列へ)
    ## Create Annotation(YOLO)
    with open(row.label_path, 'w') as f:
        if num_bbox<1:
            annot = ''
            f.write(annot)
            cnt+=1
            continue
        bboxes_voc  = coco2voc(bboxes_coco, image_height, image_width)
        bboxes_voc  = clip_bbox(bboxes_voc, image_height, image_width)
        bboxes_yolo = voc2yolo(bboxes_voc, image_height, image_width).astype(str)
        all_bboxes.extend(bboxes_yolo.astype(float)) # all_bboxesにbboxes_yoloを追加
        bboxes_info.extend([[row.image_id, row.video_id, row.sequence]]*len(bboxes_yolo)) # bboxes_infoにbboxの数だけ[image_id, video_id, sequence]を追加
        annots = np.concatenate([labels, bboxes_yolo], axis=1) # labelsの横にbboxes_yoloをくっつける(bboxの数だけ行ができる)
        string = annot2str(annots) # annotationをstrにしてる
        f.write(string)
print('Missing:',cnt)

指定した場所`kaggle/imageとかlabel`にファイルができないぞ？？？

# 📁 Create Folds
> Number of samples aren't same in each fold which can create large variance in **Cross-Validation**.

> 各フォールドのサンプル数が同じでないため、クロスバリデーションで大きなばらつきが生じる可能性があります。

GroupKFold: 同じグループが異なる分割パターンに出現しないようにデータセットを分割する。
参考：https://upura.hatenablog.com/entry/2018/12/04/224436

> クラスとは別の概念として、データセット全体は均等な10グループに分割されています。グループはなかなかイメージが付きづらいかもしれませんが、例えば「同じユーザのデータを一つのグループにまとめておく」といった使い方が想定できます。**同じユーザのデータがtrainのデータセットとvalidationのデータセットの両者に存在すると、不当に精度が高くなる恐れがある**ためです。

今回は動画の数(`len(df['sequence'].unique())`の事)

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold.html#sklearn.model_selection.GroupKFold

> class sklearn.model_selection.GroupKFold(n_splits=5)[source]
>
> オーバーラップしないグループを持つK-foldイテレータの変形。
>
> 同じグループが2つの異なるフォールドに現れることはありません（異なるグループの数は、少なくともフォールドの数と同じでなければなりません）。
>
> 各フォールドは、それぞれのフォールドで異なるグループの数がほぼ同じという意味で、ほぼバランスが取れています。

In [ ]:
from sklearn.model_selection import GroupKFold
kf = GroupKFold(n_splits = 5) # n_split: train,valのパターンの数。元データを5パターンのtrain,valに分ける
df = df.reset_index(drop=True)
df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(kf.split(df, y = df.video_id.tolist(), groups=df.sequence)): # sequence: 動画のサブセットID(同じIDの画僧は同じ動画)
    df.loc[val_idx, 'fold'] = fold
display(df.fold.value_counts())

# ⚙️ Configuration
The dataset config file requires
1. The dataset root directory path and relative paths to `train / val / test` image directories (or *.txt files with image paths)
2. The number of classes `nc` and 
3. A list of class `names`:`['cots']`

データセット設定ファイルには、以下のものが必要です。
1. 1. データセットのルートディレクトリのパスと，`train / val / test` 画像ディレクトリの相対パス (または画像パスを含む *.txt ファイル)
2. クラス数 `nc` と 
3. クラス名`:`['cots']`のリスト。

In [ ]:
train_files = []
val_files   = []
train_df = df.query("fold!=@FOLD")
valid_df = df.query("fold==@FOLD")
train_files += list(train_df.image_path.unique())
val_files += list(valid_df.image_path.unique())
len(train_files), len(val_files)

# **Data Augmentation**

train_dfに対してDAをかける

今回はbboxの位置が変わる処理はしない(label.txtを流用したいため)

できた画像は`working/images/{video_id}-{video_frame}-aug.jpg`に入れる。

その画像に対するラベルは元画像のlabel.txtから流用`working/labels/{video_id}-{video_frame}-aug.txt`に入れる。

train_dfに行を追加。(元画像の行をコピー。image_path, label_pathを↑のものに変えればOKのはず)

In [ ]:
import albumentations as A

In [ ]:
class HE_HSV(A.ImageOnlyTransform):
    def __init__(self, p: float = 0.5, always_apply=False):
        super().__init__(always_apply, p)
        
    def apply(self, image,**params):
        img_hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

        # Histogram equalisation on the V-channel
        img_hsv[:, :, 2] = cv2.equalizeHist(img_hsv[:, :, 2])

        # convert image back from HSV to RGB
        image_hsv = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2RGB)

        return image_hsv

In [ ]:
class AUG_DATASET(Dataset):
    
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform
        
    def coco2yolo(self, bboxes, image_height=720, image_width=1280):
        """
        coco => [xmin, ymin, w, h]
        yolo => [xmid, ymid, w, h] (normalized)
        """
        bboxes = np.array(bboxes)
        bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int

        # normolizinig
        bboxes[..., [0, 2]]= bboxes[..., [0, 2]]/ image_width
        bboxes[..., [1, 3]]= bboxes[..., [1, 3]]/ image_height

        # converstion (xmin, ymin) => (xmid, ymid)
        bboxes[..., [0, 1]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]/2

        return bboxes
    
    def coord_to_box(self, bouding_box, image):
        box_yolo_format = []
        height, width = image.shape[0], image.shape[1]
        
        if False: # CFG.use_coco2yolo
            box_yolo_format = self.coco2yolo(bouding_box)
            box_yolo_format = np.clip(box_yolo_format,0,1)
            label = np.repeat([0],box_yolo_format.shape[0]).reshape(-1,1)
            box_yolo_format = np.append(box_yolo_format,label, axis=1)
        else:
            for bb in bouding_box:
                label = [max(0,bb[0]), max(0,bb[1]), min(bb[0]+bb[2], 1280), min(720,bb[1]+bb[3]), '0']
                bbox_albu = A.convert_bbox_to_albumentations(label, source_format='pascal_voc', rows=height, cols=width)
                bbox_yolo = A.convert_bbox_from_albumentations(bbox_albu, target_format='yolo', rows=height, cols=width, check_validity=True)
                clip_box = [np.clip(value,0,1) for value in bbox_yolo[:-1]] + [bbox_yolo[-1]]
                box_yolo_format.append(clip_box)
        return box_yolo_format

    def bbox_to_txt(self, bboxes):
        """
        Convert a list of bbox into a string in YOLO format (to write a file).
        @bboxes : numpy array of bounding boxes 
        return : a string for each object in new line: <object-class> <x> <y> <width> <height>
        """
        txt=''
        for index,l in enumerate(bboxes):
            l = [str(x) for x in l[:4]]
            l = ' '.join(l)
            txt +=  '0' +' ' + l + '\n'
        return txt

    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self,index):
        row = self.df.iloc[index]
        path = row['image_path']
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #aug_index = row['aug_index']
        list_info = path.split('/')
        image_name = list_info[-2] + '_' + list_info[-1].split('.')[0]
        box = row['bboxes']
        bounding_box = self.coord_to_box(box, img)

        if self.transform is not None:
            res = self.transform(image=img, bboxes=bounding_box)
            img = res['image']
            bounding_box = res['bboxes']
            
        box_yolo_format = self.bbox_to_txt(bounding_box)
        return img, box_yolo_format, image_name

In [ ]:
def get_transforms():
    return A.Compose([
            HE_HSV(always_apply=True) # 現在は全データに対してDAかけた前提でtrain_dfの行を追加している
            ], bbox_params=A.BboxParams(format='yolo' , min_visibility=0.4,min_area=500))

class AUGDATA　のimage_path周りの設定いじる必要あり

できた画像は`working/images/{video_id}-{video_frame}-aug.jpg`に入れる。

その画像に対するラベルは元画像のlabel.txtから流用`working/labels/{video_id}-{video_frame}-aug.txt`に入れる。

train_dfに行を追加。(元画像の行をコピー。image_path, label_pathを↑のものに変えればOKのはず)

---

最初からtrain_dfをコピーする。=tran_aug_df

train_aug_dfに対してDAをかける

同じtrainディレクトリに保存すればok

In [ ]:
train_df

In [ ]:
if AUG is not None:
    
    train_df_he = train_df.copy(deep=True)
    
    # dataloaderを使ってデータ拡張
    dataset = AUG_DATASET(train_df_he, transform = get_transforms())
    dataloader = Data.DataLoader(dataset=dataset, num_workers=WORKER, batch_size=BATCH, shuffle=False, drop_last=False,\
                               pin_memory = False)
    
    for aug_img, aug_box, image_name in progress_bar(dataloader):
        for idx, image in enumerate(aug_img):
            name = image_name[idx]
            new_name = "{}_HE".format(name)
            image = aug_img[idx]
            box = aug_box[idx]
            
            path_txt = LABEL_DIR + "/" + new_name + ".txt"
            path_jpg = IMAGE_DIR + "/" + new_name + ".jpg"
            is_path = os.path.exists(path_jpg)
            image = image.numpy()
            cv2.imwrite(path_jpg, image[...,::-1])
            txt_file = open(path_txt, "w")
            txt_file.write(box)
            txt_file.close()
        break
    
    # train_dfにDAした行を追加
    func_he_path = lambda x: '{}_HE'.format(x)
    train_df_he.image_path = train_df_he.image_path.map(func_he_path) # image_pathを更新
    train_df_he.label_path = train_df_he.label_path.map(func_he_path) # label_pathを更新
    train_df = pd.concat([train_df, train_df_he], axis=0, ignore_index=True) #index再度降り直し

In [ ]:
len(train_df.image_path.unique())

## train.txt, val.txtを作る(学習と検証データのパスが記述されている)

In [ ]:
import yaml

cwd = '/kaggle/working/'

with open(os.path.join( cwd , 'train.txt'), 'w') as f:
    for path in train_df.image_path.tolist():
        f.write(path+'\n')
            
with open(os.path.join(cwd , 'val.txt'), 'w') as f:
    for path in valid_df.image_path.tolist():
        f.write(path+'\n')

data = dict(
    path  = '/kaggle/working',
    train =  os.path.join( cwd , 'train.txt') ,
    val   =  os.path.join( cwd , 'val.txt' ),
    nc    = 1,
    names = ['cots'],
    )

with open(os.path.join( cwd , 'gbr.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(os.path.join( cwd , 'gbr.yaml'), 'r')
print('\nyaml:')
print(f.read())

In [ ]:
%%writefile /kaggle/working/hyp.yaml
lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.10  # image translation (+/- fraction)
scale: 0.5  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.5  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 0.5  # image mosaic (probability)
mixup: 0.5 # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)

# 📦 [YOLOv5](https://github.com/ultralytics/yolov5/)

In [ ]:
%cd /kaggle/working
!rm -r /kaggle/working/yolov5
# !git clone https://github.com/ultralytics/yolov5 # clone
!cp -r /kaggle/input/yolov5-lib-ds /kaggle/working/yolov5
%cd yolov5
%pip install -qr requirements.txt  # install

from yolov5 import utils
display = utils.notebook_init()  # check

# 🚅 Training

In [ ]:
!python train.py --img {DIM}\
--batch {BATCH}\
--epochs {EPOCHS}\
--data /kaggle/working/gbr.yaml\
--hyp /kaggle/working/hyp.yaml\
--weights {MODEL}.pt\
--optimizer {OPTIM}\
--project {PROJECT} --name {NAME}\
--exist-ok

# ✨ Overview

## Output Files

In [ ]:
OUTPUT_DIR = '{}/{}'.format(PROJECT, NAME)
!ls {OUTPUT_DIR}

# 📈 Class Distribution

In [ ]:
plt.figure(figsize = (10,10))
plt.axis('off')
plt.imshow(plt.imread(f'{OUTPUT_DIR}/labels_correlogram.jpg'));

In [ ]:
plt.figure(figsize = (10,10))
plt.axis('off')
plt.imshow(plt.imread(f'{OUTPUT_DIR}/labels.jpg'));

# 🔭 Batch Image

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (10, 10))
plt.imshow(plt.imread(f'{OUTPUT_DIR}/train_batch0.jpg'))

plt.figure(figsize = (10, 10))
plt.imshow(plt.imread(f'{OUTPUT_DIR}/train_batch1.jpg'))

plt.figure(figsize = (10, 10))
plt.imshow(plt.imread(f'{OUTPUT_DIR}/train_batch2.jpg'))

## GT Vs Pred

In [ ]:
fig, ax = plt.subplots(3, 2, figsize = (2*9,3*5), constrained_layout = True)
for row in range(3):
    ax[row][0].imshow(plt.imread(f'{OUTPUT_DIR}/val_batch{row}_labels.jpg'))
    ax[row][0].set_xticks([])
    ax[row][0].set_yticks([])
    ax[row][0].set_title(f'{OUTPUT_DIR}/val_batch{row}_labels.jpg', fontsize = 12)
    
    ax[row][1].imshow(plt.imread(f'{OUTPUT_DIR}/val_batch{row}_pred.jpg'))
    ax[row][1].set_xticks([])
    ax[row][1].set_yticks([])
    ax[row][1].set_title(f'{OUTPUT_DIR}/val_batch{row}_pred.jpg', fontsize = 12)
plt.show()

# 🔍 Result

## Score Vs Epoch

In [ ]:
plt.figure(figsize=(30,15))
plt.axis('off')
plt.imshow(plt.imread(f'{OUTPUT_DIR}/results.png'));

## Confusion Matrix

In [ ]:
plt.figure(figsize=(12,10))
plt.axis('off')
plt.imshow(plt.imread(f'{OUTPUT_DIR}/confusion_matrix.png'));

## Metrics

In [ ]:
for metric in ['F1', 'PR', 'P', 'R']:
    print(f'Metric: {metric}')
    plt.figure(figsize=(12,10))
    plt.axis('off')
    plt.imshow(plt.imread(f'{OUTPUT_DIR}/{metric}_curve.png'));
    plt.show()

## Please Upvote if you find this Helpful

# ✂️ Remove Files

In [ ]:
!rm -r {IMAGE_DIR}
!rm -r {LABEL_DIR}

<img src="https://www.pngall.com/wp-content/uploads/2018/04/Under-Construction-PNG-File.png">